In [ ]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

import pandas as pd
from scipy.interpolate import make_interp_spline

In [ ]:
# filename
# period
# t0
fname = ''
P = 9.9740514
t0 = 2830.1961049

lc_s056 = pd.read_table(fname, delimiter='\t')

time = lc_s056.time.values
flux = lc_s056.flux.values
flux_err = lc_s056.flux_err.values
qual = lc_s056.quality.values

data = np.zeros((len(time[qual == 0]), 3))
data[:,0], data[:,1], data[:,2] = time[qual ==  0], flux[qual == 0], flux_err[qual == 0]

In [ ]:
# template - accounting for the eclipsed data
template_sin = np.genfromtxt('../data/template_s056_qua')

template_sin[:,1] /= np.median(template_sin[:,1])


# template - binary model
template_bin = np.genfromtxt('../data/bintemplate_s056_qua')

template_bin[:,1] /= np.median(template_bin[:,1])

In [ ]:
def convert(time, P, t0):
    result = (time - t0) / P
    return result

data[:,0] = convert(time = data[:,0], P = P, t0 = t0)

In [ ]:
# limits of eclipse:
ecl_limit = [[-0.021,0.021],[0.48,-0.47]]

def phase(input):
    result = np.copy(input)
    result += 1.
    result = result % 1
    result -= 1.
    return result

t_phase = phase(data[:,0])

mask = ((t_phase > ecl_limit[0][0]) & (t_phase < ecl_limit[0][1])) | (t_phase > ecl_limit[1][0]) | (t_phase < ecl_limit[1][1])
mask = ~mask

In [ ]:
# configuration for binning:

bin_from = 100
bin_to = 121
bin_step = 2

n_bins = (bin_to - bin_from) // 2 + 1
print('number of interpolations: {0:d}'.format(n_bins))

y_intps = np.zeros((len(data[:,0]), n_bins))
i_int = 0
for bin_try in range(bin_from, bin_to, bin_step):
	t_s = data[mask,0][bin_try // 2::bin_try]
	y_s = np.zeros(len(t_s))
	for i in range(len(t_s)):
		y_s[i] = np.median(data[mask, 1][0+i*bin_try:bin_try+i*bin_try])

	bspl = make_interp_spline(t_s, y_s, k=3)
	y_intps[:,i_int] = bspl(data[:,0])
	
	i_int += 1

y_s = np.median(y_intps, axis=1)

In [ ]:
data_rawpuls = np.copy(data[:, 1] / (template_bin[:,1] * y_s))


bin_from = 100
bin_to = 121
bin_step = 2

n_bins = (bin_to - bin_from) // 2 + 1
print('number of interpolations: {0:d}'.format(n_bins))
y_intps = np.zeros((len(data[:,0]), n_bins))
i_int = 0
for bin_try in range(100,121,2):
	t_s3 = data[:,0][bin_try // 2::bin_try]
	y_s3 = np.zeros(len(t_s3))
	for i in range(len(t_s3)):
		y_s3[i] = np.median(data_rawpuls[0+i*bin_try:bin_try+i*bin_try])

	bspl = make_interp_spline(t_s3, y_s3, k=3)
	y_intps[:,i_int] = bspl(data[:,0])
	i_int += 1

y_s3s = np.median(y_intps, axis=1)

In [ ]:
y_s4s = y_s * y_s3s

data_rawpuls_r2 = np.copy((data[:, 1] - (template_bin[:,1] * y_s4s)) / np.median(y_s4s))

In [ ]:
# If you want to check it, uncomment it

# plt.figure(figsize=(12,5))

# plt.plot(data[:,0], data[:,1] / np.median(data[:,1]), 'o', color = 'gray', ms = 0.5)
# plt.plot(data[:,0], data_rawpuls_r2 + 1, 'ko', ms = 0.5)

# plt.xlim(-0.5,0.5)
# plt.ylim(0.9,1.1)
# plt.show()


# plt.figure(figsize=(12,5))

# plt.plot(data[:,0], data[:,1] / np.median(data[:,1]), 'o', color = 'gray', ms = 0.5)
# plt.plot(data[:,0], data_rawpuls_r2 + 1, 'ko', ms = 0.5)

# plt.xlim(0.5,1.5)
# plt.ylim(.9,1.1)
# plt.show()

# plt.figure(figsize=(12,5))

# plt.plot(data[:,0], data[:,1] / np.median(data[:,1]), 'o', color = 'gray', ms = 0.5)
# plt.plot(data[:,0], data_rawpuls_r2 + 1, 'ko', ms = 0.5)

# plt.xlim(1.5,2.5)
# plt.ylim(.9,1.1)
# plt.show()

In [ ]:
# exporting it
data_export = np.zeros((len(data[:,0]),3))

data_export[:,0] = data[:,0]
data_export[:,1] = data_rawpuls_r2
data_export[:,2] = data[:,2] / y_s4s

np.savetxt('../data/tess_gk_dra_s056.norm', data_export)